<img src=images/widsatx-logo-outline.png align="left" width="18%">

<h1 align='center'>MapReduce, etc.</h1>
<h3 align='center'>June 15, 2017</h3>
<h3 align='center'>Women in Data Science - ATX</h3>
<h3 align='center'>Data Science from Scratch Workshop</h3>

## Question: What sorts of computational blockers have you come across?

## Some Solutions
#### CPU Bound
    - Optimize algorithms
    - Optimize compilation
    - Parallelize code
#### I/O Bound
    - Decrease network transfer time
        - Compress data for faster transfer
        - Move computation to the data
    - Consider multithreading
        - Unblock code by allowing data to transfer in a separate thread
#### Storage Bound
    - Store data elsewhere and transfer to compute machine when needed
    - Spread data across multiple machines and run computations on all machines
#### Memory Bound
    - Cache data for faster access

# Overview of Workshop
- **MapReduce:** What and Why
    - Examples:
        - Word Count
        - Analyzing Status Updates
        - Matrix Multiplier
    - References and Resources
- **Hadoop:** What and Why
- **(Py)Spark:** What and Why
    - Examples:
        - Word Count
        - Logistic Regression
        - Clickstream
    - References and Resources

# MapReduce: What and Why

## What is MapReduce?

(Material in the slides on MapReduce was taken from Chapter 24: MapReduce in "Data Science from Scratch" by J. Grus)

Basic version of `MapReduce` algorithm:
1. User a `mapper` function to turn each item into zero or more key-value
pairs.
2. Collect together all the pairs with identical keys.
3. User a `reducer` function on each collection of grouped values to produce output values for the corresponding key.

## Why MapReduce?

"The primary benefit of MapReduce is that it allows us to distribute computations by moving the processing to the data."
-- J. Grus

### Example: Word Count

In [ ]:
from tokenize import tokenize

def word_count_old(documents):
    """word count not using MapReduce"""
    return Counter(word
        for document in documents
        for word in tokenize(document))

In [ ]:
def wc_mapper(document):
    """for each word in document, emit (word,1)"""
    for word in tokenize(document):
        yield (word, 1)
        
def wc_reducer(word, counts):
    """sum up the counts for a word"""
    yield (word, sum(counts))

In [ ]:
from collections import defaultdict

def word_count(documents):
    """count the words in the input documents using MapReduce"""
    
    # place to store grouped values
    collector = defaultdict(list)
    
    for document in documents:
        for word, count in wc_mapper(document):
            collector[word].append(count)
            
    return [output
            for word, counts in collector.iteritems()
            for output in wc_reducer(word, counts)]

In [ ]:
def map_reduce(inputs, mapper, reducer):
    """runs MapReduce on the inputs using mapper and reducer"""
    collector = defaultdict(list)
    
    for input in inputs:
        for key, value in mapper(input):
            collector[key].append(value)
            
    return [output
           for key, values in collector.iteritems()
           for output in reducer(key, values)]

In [ ]:
# TODO: Replace this with input file
documents = ['foo', 'bar', 'baz', 'data', 'science']

In [ ]:
word_counts = map_reduce(documents, wc_mapper, wc_reducer)

### Example: Analyzing Status Updates

### Example: Matrix Multiplier

# Resources and References (MapReduce)
- "Data Science From Scratch" - J. Grus

### For Futher Exploration
- The most widely used MapReduce system is Hadoop (http://hadoop.apache.org), which itself merits many books.  There are various commercial and noncommercial distributions and a huge ecosystem of Hadoop-related tools.
    In order to use it, you have to set up your own *cluster* (or find someone to let you use theirs), which is not necessarily a task for the faint-hearted.  Hadoop mappers and reducers are commonly written in Java, although there is a facility known as "Hadoop streaming" that allows you to write them in other languages (including Python).

- Amazon.com offers an Elastic MapReduce (http://aws.amazon.com/elasticmapreduce/) service that can programmatically create and destroy clusters, charging only for the amount of time that you're using them.

- mrjob (https://github.com/Yelp/mrjob) is a Python package for interfacing with Hadoop (or Elastic MapReduce).

- Hadoop jobs are typically high-latency, which makes them a poor choice for "real-time" analytics.  There are various "real-time" tools built on top of Hadoop, but there are also several alternative frameworks that are growing in popularity.  Two of the most popular are Spark (http://spark.apache.org/) and Storm (http://storm.incubator.apache.org/).


# Hadoop: What and Why

## What is Hadoop?


## Why Hadoop?

### Example: MapReduce for Hadoop Streaming

https://github.com/jddavis-100/Statistics-and-Machine-Learning/blob/gh-pages/MapReduce_Word_Count

In [ ]:
"""
This code is an outline/example of how to write a map reduce program in python.
This is helpful for writing a simple MapReduce program for streaming on Hadoop. 
For more information on MapReduce and Hadoop Streaming,
see the wiki page titled Hadoop Streaming and MapReduce. Other scripts are uploaded sepparately.
General references to learn about Hadoop streaming and python MapReduce jobs will be listed on the wiki.

"""

import sys

#input comes from STDIN (standard input)
#feeding in lines and then counting words

for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # split the line into words
    words = line.split()
    # increase counters
    for word in words:
        # write the results to STDOUT (standard output);
        # what we output here will be the input for the
        # Reduce step, i.e. the input for reducer.py
        # tab-delimited; the trivial word count is 1
        print('{}\t{}'.format(word, 1)
  

In [ ]:
from operator import itemgetter
import sys

current_word = None
current_count =0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    
    #parse the input we got form mapper.py
    word, count = line.split('\t', 1)
    
    #convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        #count was not a number, so silently
        #ignore/discard this line
        continue
    
    #this IF-switch only works because Hadoop sorts map output
    #by key (here:word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            #write result to STDOUT
            print('{}\t{}'.format(current_word, current_count))
        current_count = count
        current_word = word
  
#do not forget to output the last word if needed!
if current_word == word:
     print('{}\t{}'.format(current_word, current_count))

# Spark (PySpark)

# Environment Setup
We'll use the free Databricks Comunity Edition platform to run our Spark jobs: 
1. Use Google Chrome browser (Firefox should also work, but not Internet Explorer, Safari, etc.)
2. Sign up for the Community Edition account here: https://databricks.com/try-databricks

Or, feel free to use a local installation of Spark, etc. If Spark isn't already installed on your machine it can take up to an hour to download and build from source locally (there are also pre-built versions that would be faster to set up):
1. Download: http://spark.apache.org/downloads.html
2. Open: `../spark1.6.1/README.md`
3. Build: `../spark1.6.1/build/mvn -DskipTests clean package`
    - On my laptop, time to build was ~30 mins.

# Spark: What and Why

## What is Spark?
Spark is a fast and expressive cluster computing system for doing Big Data computation. It's good for iterative tasks, for doing big batch processing, and for interactive data exploration.

It's compatible with Hadoop-supported file systems and data formats (HDFS, S3, SequenceFile, ...), so if you've been using Hadoop you can use it with your existing data and deploy it on your existing clusters.

It achieves fault tolerance through *lineage*: if you lose a partition (chunk) of data you can reconstruct it through a set of *transformations* that act on data stored in memory. This is in contrast to distributed shared memory systems where you have to write to disk and roll back.

## Why use Spark?
- Speed
- Ease of Use
- Generality
- Runs Everywhere


>"Although current frameworks provide numerous abstractions for accessing a cluster’s computational resources, they lack abstractions for leveraging distributed memory. This makes them inefficient for an important class of emerging applications: those that reuse intermediate results across multiple computations. Data reuse is common in many <font color='blue'>iterative</font> machine learning and graph algorithms, including PageRank, K-means clustering, and logistic regression. Another compelling use case is <font color='blue'>interactive</font> data mining, where a user runs multiple ad-hoc queries on the same subset of the data. Unfortunately, in most current frameworks, the only way to reuse data between computations (e.g., between two MapReduce jobs) is to write it to an external stable storage system, e.g., a distributed file system. This incurs substantial overheads due to data replication, disk I/O, and serialization, which can dominate application execution times."

- Zaharia et al., "Resilient Distributed Datasets: A Fault-Tolerant Abstraction for In-Memory Cluster Computing," *In NSDI '12*, April 2012

## Spark vs MapReduce vs MPI vs ...
- [MapReduce](https://en.wikipedia.org/wiki/MapReduce) --> [Hadoop](http://hadoop.apache.org/): heavily used in business computing
- [Message Passing Interface (MPI)](https://en.wikipedia.org/wiki/Message_Passing_Interface) --> [MVAPICH](http://mvapich.cse.ohio-state.edu/): heavily used in scientific computing
- [Spark](http://spark.apache.org/): complement to Hadoop, faster for iterative applications, rich set of APIs in Scala, Python, and Java, and an interactive shell

## Spark Architecture
- Spark Driver and Workers
- SparkContext (replaced by SparkSession in version 2.X)
- Spark applications run as independent sets of processes on a cluster, coordinated by the SparkContext object in your main program (called the driver program).
- SparkContext can connect to several types of cluster managers (either Spark’s own standalone cluster manager, Mesos or YARN)

<img src=images/cluster-overview.png align="center" width="75%">

<h4 align='right'>https://spark.apache.org/docs/1.1.0/cluster-overview.html</h4>

## Spark Programming Concepts (version 1.X)
- ****SparkContext****: entry point to Spark functions
- ****Resilient Distributed Datasets (RDDs)****:
    - Immutable, distributed collections of objects
    - Can be cached in memory for fast reuse
- ****Operations on RDDs****:
    - *Transformations*: define a new RDD (map, join, ...)
    - *Actions*: return or output a result (count, save, ...)
- ****Two ways to create RDDs****:
    1. By parallelizing an existing collection in your driver program:  
        `data = [1, 2, 3, 4, 5]  
        distData = sc.parallelize(data)`  
    2. Or by referencing a dataset in an external storage system, such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat:  
        `distFile = sc.textFile("data.txt")`       
<h4 align='right'>http://spark.apache.org/docs/latest/programming-guide.html</h4>

## Spark Data Interfaces (versions 1.X and 2.X)

There are several key interfaces that you should understand when you go to use Spark.

-   ****The Dataset****
    -   The Dataset is Apache Spark's newest distributed collection and can be considered a combination of DataFrames and RDDs. It provides the typed interface that is available in RDDs while providing a lot of conveniences of DataFrames. It will be the core abstraction going forward.
-   ****The DataFrame****
    -   The DataFrame is collection of distributed `Row` types. These provide a flexible interface and are similar in concept to the DataFrames you may be familiar with in python (pandas) as well as in the R language.
-   ****The RDD (Resilient Distributed Dataset)****
    -   Apache Spark's first abstraction was the RDD or Resilient Distributed Dataset. Essentially it is an interface to a sequence of data objects that consist of one or more types that are located across a variety of machines in a cluster. RDD's can be created in a variety of ways and are the "lowest level" API available to the user. While this is the original data structure made available, new users should focus on Datasets as those will be supersets of the current RDD functionality.

*(slide taken from "Introduction to Apache Spark on Databricks" notebook)*

## What is PySpark?
- The Python API for Spark
- Run interactive jobs in the shell
- Supports numpy, pandas and other Python libraries

## Why use PySpark?
- If you already know Python
- Can use Spark in tandem with your favorite Python libraries
- If you don't need Python libraries, maybe just write code in Scala

### PySpark's core classes (version 1.X):
- ****pyspark.SparkContext****  
Main entry point for Spark functionality.
- ****pyspark.RDD****  
A Resilient Distributed Dataset (RDD), the basic abstraction in Spark.
- ****pyspark.streaming.StreamingContext****  
Main entry point for Spark Streaming functionality.
- ****pyspark.streaming.DStream****  
A Discretized Stream (DStream), the basic abstraction in Spark Streaming.
- ****pyspark.sql.SQLContext****  
Main entry point for DataFrame and SQL functionality.
- ****pyspark.sql.DataFrame****  
A distributed collection of data grouped into named columns.

## Transformations
- Transform one RDD to another, **new** RDD (immutable)

| Transformation | Description | Type |
| :------:  | :-----------: | :-----: |
| `map(func)`     | Apply a function over each element | Narrow |
| `flatMap(func)` | Map then flatten output | Narrow |
| `filter(func)`  | Keep only elements where function is `True` | Narrow |
| `sample(withReplacement, fraction, seed)` | Return a sampled subset of this RDD | Narrow |
| `groupByKey(k, v)` | Group the values for each key in the RDD into a single sequence | Wide |
| `reduceByKey(func)` | Merge the values for each key using an associative reduce function | Wide |


<img src=images/narrow_wide_transformations.png align="center" width="100%">

<h4 align='right'>https://dzone.com/articles/big-data-processing-spark</h4>

## Actions
- Return or output a result

| Action | Description | Try it Out\*|
| :------:  | :-----------:| :---: |
| `collect()`     | Return a list that contains all of the elements in this RDD | `sc.parallelize([0, 2, 3, 4, 6], 5).glom().collect()` |
| `count()`  | Return the number of elements | `sc.parallelize([2, 3, 4]).count()` |
| `saveAsTextFile(path)` | Save as a text file, using string representations of elements | `sc.parallelize(['foo', '-', 'bar', '!']).saveAsTextFile("/FileStore/foo-bar.txt")])`|
| `first()`    | Return the first element | `sc.parallelize([2, 3, 4]).first()` |
| `take(num)`    | Take the first num elements | `sc.parallelize([2, 3, 4, 5, 6]).take(2)` |

### \* Try it Out:
1. Go to your databricks Workspace and create a new directory within your Users directory called "2016-06-20-pyladies-pyspark" 
2. Create a notebook called "0-Introduction"  within this directory
3. Type or copy/paste lines of code into separate cells and run them (you will be prompted to launch a cluster) 

When using Databricks the `SparkContext` is created for you automatically as `sc`.

In the Databricks Community Edition there are no Worker Nodes - the Driver Program (Master) executes the entire code.

### Try a couple more examples with transformations and actions:

In [ ]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
sorted(rdd.groupByKey().mapValues(len).collect())
sorted(rdd.groupByKey().mapValues(list).collect())

In [ ]:
from operator import add

rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
sorted(rdd.reduceByKey(add).collect())

We've shown only a subset of possible *transformations* and *actions*. Check out others for your application in the docs: http://spark.apache.org/docs/latest/api/python/pyspark.html

### Example: Log Mining

In [ ]:
val lines = spark.textFile("hdfs://...")
val errors = lines.filter(_.startsWith("ERROR"))
val messages = errors.map(_.split('\t')(2))

messages.filter(_.contains("foo")).count

The computation is expressed declaratively and nothing actually takes place until calling `count` at the end.

# Exercises

## Exercise 1: Word Count
Create a few transformations to build a dataset of (String, Int) pairs called counts and then save it to a file.

1. Create a notebook in "2016-06-20-pyladies-pyspark" called "1-WordCount"
2. Try to implement the following Word Count example:

http://spark.apache.org/examples.html

In [ ]:
text_file = sc.textFile("hdfs://...")
counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)
counts.saveAsTextFile("hdfs://...")

## Exercise 2: Logistic Regression

1. Create a notebook in "2016-06-20-pyladies-pyspark" called "2-LogisticRegression"
2. Try to implement one of the following Logistic Regression examples:
    - http://spark.apache.org/examples.html (Prediction with Logistic Regression)
    - https://github.com/apache/spark/blob/master/examples/src/main/python/mllib/logistic_regression.py
    - https://github.com/apache/spark/blob/master/examples/src/main/python/logistic_regression.py

In [ ]:
# Every record of this DataFrame contains the label and
# features represented by a vector.
df = sqlContext.createDataFrame(data, ["label", "features"])

# Set parameters for the algorithm.
# Here, we limit the number of iterations to 10.
lr = LogisticRegression(maxIter=10)

# Fit the model to the data.
model = lr.fit(df)

# Given a dataset, predict each point's label, and show the results.
model.transform(df).show()

### Exercise 3: Clickstream

1. Create a notebook in "2016-06-20-pyladies-pyspark" called "3-Clickstream"
2. Implement the Clickstream example from the Databricks "Quick Start DataFrames" notebook in Python
3. Or just run through the example in Scala

# Resources and References (Spark)
- ****MOOCs****:
    - "Introduction to Apache Spark": https://www.edx.org/course/introduction-apache-spark-uc-berkeleyx-cs105x
    - "Hadoop Platform and Application Framework" (week 5 covers Spark): https://www.coursera.org/learn/hadoop/home/week/5
- ****Spark/PySpark Docs****:
    - (v1.6.1 - latest) http://spark.apache.org/docs/latest/
        - http://spark.apache.org/docs/latest/api/python/
    - (v2.0.0) http://spark.apache.org/docs/2.0.0-preview/
    - http://spark.apache.org/research.html
    - http://spark.apache.org/examples.html
- ****Other****:
    - Spark 2.0 Webinar (2016): http://go.databricks.com/apache-spark-2.0-presented-by-databricks-co-founder-reynold-xin
    - PySpark Talk (J. Rosen, 2013): https://www.youtube.com/watch?v=xc7Lc8RA8wE

# Thanks for Coming!